In [1]:
search_for = 136
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.08453774452209473
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 1048119951
type: [1, 1, 1, 1, 136] 136
cases of this type: 342102016
100000 30.420475055716057
200000 50.520419669898025
300000 55.64720638289716
400000 55.08119263885785
500000 58.24363624499809
600000 57.851811179333886
700000 58.38544059260447
800000 63.47018587176844
900000 66.8014880885571
1000000 80.06613459473041
1100000 78.93660350293294
1200000 83.9402090844644
1300000 28.583249976305584
1400000 87.77795170140611
1500000 101.27953678301975
1600000 97.99535572819114
1700000 97.56147619834114
1800000 98.4648196932355
1900000 108.36665116063077
2000000 123.83589721668072
2100000 109.39064635407584
2200000 104.51148081105502
2300000 105.61472147316836
2400000 113.19978314461201
2500000 92.63423802002907
2600000 33.87605201620456
2700000 98.56868089151196
2800000 101.63081180667406
2900000 102.77108853369845
3000000 99.29529

29400000 105.61472818267396
29500000 105.69438147239032
29600000 100.83206296090142
29700000 103.33533471724564
29800000 104.88259065172556
29900000 104.38763182555262
30000000 67.46747730178899
30100000 86.2025148710044
30200000 116.55301084774278
30300000 111.6465887945024
30400000 93.5303847361187
30500000 58.607124633805405
30600000 113.05696705051443
30700000 114.48349631145032
30800000 117.54972437774991
30900000 108.59223834289473
31000000 117.29980281977959
31100000 110.46282219245926
31200000 107.33451925764977
31300000 114.42031975906482
31400000 107.22061221024093
31500000 20.917484873657887
31600000 112.83200941622859
31700000 107.71512793203895
31800000 108.3525557094115
31900000 111.93190315500786
32000000 108.4590249740082
32100000 107.51565261425553
32200000 110.86553276821878
32300000 120.5014483925662
32400000 110.02640727041012
32500000 62.66941345064995
32600000 89.96553464420639
32700000 113.89052106498693
32800000 104.04735497142912
32900000 112.07400268938487
330

59200000 44.408930814451345
59300000 92.33828157816657
59400000 96.41506096102405
59500000 93.16361626653527
59600000 102.7422433395354
59700000 95.34494509833198
59800000 97.8741232561508
59900000 105.6474236309604
60000000 34.316898045679835
60100000 113.59357369174272
60200000 107.65537580755647
60300000 103.06664863536625
60400000 104.9578415884456
60500000 108.9930783551444
60600000 115.56376936058639
60700000 109.47133454431213
60800000 97.52186451756437
60900000 49.48882262352041
61000000 108.35004057627789
61100000 110.28207149146614
61200000 102.20337843564218
61300000 105.85444854401747
61400000 98.99500640439145
61500000 100.3044057597994
61600000 91.49142040395748
61700000 28.621925399794012
61800000 99.9902612860462
61900000 100.1877664028681
62000000 106.07745848552521
62100000 99.78344915579129
62200000 102.10297034041723
62300000 103.93278777888175
62400000 107.37691547035901
62500000 36.066319021218355
62600000 112.69816853901368
62700000 109.76036534801295
62800000 10

89200000 90.2159222929725
89300000 64.68128890172385
89400000 55.34405116883939
89500000 85.88652470371949
89600000 89.61815089400217
89700000 89.60269699498369
89800000 92.43289114067096
89900000 77.4475795666382
90000000 58.96915278892365
90100000 92.74089894940792
90200000 91.15021127600696
90300000 98.12904211427659
90400000 95.53412374900725
90500000 103.54747577703452
90600000 97.64910666053694
90700000 104.52701929285686
90800000 106.4479588181527
90900000 105.63454118550202
91000000 101.07397708333498
91100000 102.75987645691013
91200000 90.59567754928727
91300000 45.06798218558325
91400000 94.19139669030851
91500000 96.85759657430435
91600000 98.78488844246078
91700000 93.02667471622483
91800000 76.97769625472576
91900000 40.04800663907351
92000000 92.05966575464804
92100000 95.3115610760482
92200000 91.19654824851104
92300000 93.67082545184188
92400000 71.73280832468795
92500000 60.02862597332303
92600000 100.7380431929716
92700000 97.47668657527078
92800000 103.7443952381972

118800000 90.11435898710158
118900000 92.37515238687008
119000000 83.41166423798698
119100000 61.75758503765603
119200000 57.41148530233834
119300000 88.370731625745
119400000 87.48224364115687
119500000 50.21913305951839
119600000 66.1699708323738
119700000 88.64781591141869
119800000 91.06910874196596
119900000 50.074205006078294
120000000 77.88789328395113
120100000 88.92598507582305
120200000 86.64209003867659
120300000 96.01790625107631
120400000 93.5173109437938
120500000 98.38837480260553
120600000 108.40658920808193
120700000 96.70579109899522
120800000 98.01870340461222
120900000 103.88468966613905
121000000 101.29670106718088
121100000 108.67528567634947
121200000 105.53164259824248
121300000 92.30206721881838
121400000 94.17959431770375
121500000 86.75553894619055
121600000 89.03879360025525
121700000 32.43713702435183
121800000 98.02901706579918
121900000 91.40438667986905
122000000 62.83215066938471
122100000 55.60433975033073
122200000 90.76228719187729
122300000 90.07309

...

952600000 1.2622641522352405
952700000 1.202188948443626
952800000 0.5825038802222471
952900000 1.1313471912436863
953000000 1.4343719107686819
953100000 1.8970253967287998
953200000 1.3267376872725116
953300000 1.355495455896743
953400000 0.9054931449243725
953500000 0.7678562735711435
953600000 1.5422282053283423
953700000 1.426300198402951
953800000 1.3530820554892706
953900000 1.2877485613144717
954000000 0.9345235391012451
954100000 0.5786100578784347
954200000 1.347527407576922
954300000 1.399998113489771
954400000 1.7680474707610925
954500000 1.478782592775587
954600000 1.2262483346885742
954700000 0.545294612376533
954800000 1.2123888777978393
954900000 1.4250867852146347
best so far: 0
type: [1, 2, 1, 17, 4] 136
cases of this type: 2515456
955000000 1.5791275368002144
955100000 1.983029060503069
955200000 1.1865550143489718
955300000 1.0774380023119072
955400000 1.5971748981913458
955500000 1.1104405993538498
955600000 1.1861850707640555
955700000 1.3464364954098178
955800000 

best so far: 0
type: [1, 2, 4, 17, 1] 136
cases of this type: 157216
979100000 0.47240056643178524
best so far: 0
type: [1, 2, 17, 1, 4] 136
cases of this type: 147968
979200000 0.31523840720297697
979300000 0.35037656346577006
best so far: 0
type: [1, 2, 17, 2, 2] 136
cases of this type: 73984
979400000 0.31656896308075255
best so far: 0
type: [1, 2, 17, 4, 1] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 1, 2] 136
cases of this type: 36992
best so far: 0
type: [1, 2, 34, 2, 1] 136
cases of this type: 18496
best so far: 0
type: [1, 2, 68, 1, 1] 136
cases of this type: 9248
979500000 0.30524098763368723
best so far: 0
type: [1, 4, 1, 1, 34] 136
cases of this type: 5345344
979600000 1.6655182590624853
979700000 1.7388584802445448
979800000 3.0084714588833035
979900000 1.9814239219950556
980000000 1.5163293777970905
980100000 1.8735675961304543
980200000 1.3714364077564591
980300000 1.3096009322957674
980400000 1.1484215872357069
980500000 1.3055682266356283
980600000 1.1

998100000 0.8478221357631743
998200000 1.5100962528196684
998300000 2.2456406259954065
998400000 0.8372636393708528
998500000 1.2436167210888969
998600000 1.6065989017408722
998700000 0.9233008546492893
998800000 1.1539067839530466
998900000 1.3530712486245657
999000000 0.9920486549501082
999100000 1.146533462893325
999200000 1.3559829785187636
999300000 1.429778095633006
999400000 1.1389509438119012
999500000 1.304797843218428
999600000 1.1349955220872898
999700000 0.8778066257491145
999800000 1.3932531420988488
999900000 1.2220464044290482
1000000000 0.7764324203763856
1000100000 1.2849264000181992
1000200000 1.3713876905193865
1000300000 0.9071368419253164
1000400000 1.3774230935644907
1000500000 0.30400740003659926
1000600000 0.24238019555101525
1000700000 0.24473678341615598
1000800000 0.2403692380908986
1000900000 0.23810250461388893
1001000000 0.352135502310933
1001100000 0.30408879669887423
1001200000 0.28380855524911147
1001300000 0.3109014890961932
1001400000 0.24102788187326

1025000000 0.3336608042136345
1025100000 0.23462666312564412
1025200000 0.2975596815297418
1025300000 0.32477977782445683
1025400000 0.2279731216467142
1025500000 0.24588089854782558
1025600000 0.3114502995773216
1025700000 0.24355591947977343
1025800000 0.22072993314999806
1025900000 0.30145345371741056
1026000000 0.26492200345487993
1026100000 0.19101665258395736
1026200000 0.3654005724091742
1026300000 0.36187925640254615
1026400000 0.15662983932177293
1026500000 0.31460627828596566
1026600000 0.3754919082225204
1026700000 0.16568366260351938
1026800000 0.3332605736731264
1026900000 0.42161660058697065
1027000000 0.18249434791991975
1027100000 0.40209741052578485
1027200000 0.5022963910169006
1027300000 0.21572644455991122
1027400000 0.2524383942976203
1027500000 0.2870407082466622
1027600000 0.2182478308924357
1027700000 0.22764719416491122
best so far: 0
type: [2, 1, 1, 17, 4] 136
cases of this type: 1257728
1027800000 0.30142003012457486
1027900000 0.2582097069792774
1028000000 0

1045400000 0.040066670123106905
1045500000 0.05093242677463492
1045600000 0.04531792974068721
1045700000 0.04072227805062731
1045800000 0.045016041990963616
1045900000 0.03517129533509082
1046000000 0.02810195985453725
1046100000 0.010402793647563458
1046200000 0.029349797468555636
1046300000 0.03435668480249577
1046400000 0.04001679724345605
1046500000 0.028439389451698462
1046600000 0.02682995710297492
best so far: 0
type: [4, 1, 1, 2, 17] 136
cases of this type: 668168
1046700000 0.02828222389703989
1046800000 0.014766686624590554
1046900000 0.01351447183227539
1047000000 0.010541642550355196
1047100000 0.005179427864085303
1047200000 0.008439917336170541
1047300000 0.00965877493841648
best so far: 0
type: [4, 1, 1, 17, 2] 136
cases of this type: 78608
1047400000 0.006017415580215719
best so far: 0
type: [4, 1, 1, 34, 1] 136
cases of this type: 39304
best so far: 0
type: [4, 1, 2, 1, 17] 136
cases of this type: 334084
1047500000 0.0037014785675221016
1047600000 0.006365081381360691
